In [17]:
#!pip install shap optuna
#from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
import shap


import optuna
from sklearn.model_selection import train_test_split
from optuna.integration import LightGBMPruningCallback

#Reading the csv data into a variable. CSV was stored on Google Drive.
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/train.csv")

#Removing the non-number columns from df. Storing all but last columns in x, last column in y
df_int = df._get_numeric_data()
x=df_int.drop("SalePrice",axis=1)
y=df_int.SalePrice
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

#Tuning the hyperparameters with Optuna
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        'metric': 'rmse',
        "n_estimators": trial.suggest_categorical("n_estimators", [20000]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 10, 100),
        #"min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.9, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.9, step=0.1
        ),
        #'min_child_samples': trial.suggest_int('min_child_samples', 1, 150),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    olgbm = lgbm.LGBMRegressor(**param)
    olgbm.fit(X_train, y_train,eval_set=[(X_test,y_test)], callbacks=[lgbm.early_stopping(100), lgbm.log_evaluation, LightGBMPruningCallback(trial,"rmse")])
    predictions = olgbm.predict(X_test)
    accuracy = mean_squared_error(y_test, predictions)
    return accuracy

study = optuna.create_study(study_name="lightgbm", direction="minimize")
study.optimize(objective, n_trials=50)

model = lgbm.LGBMRegressor(**study.best_params)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

fig1 = optuna.visualization.plot_optimization_history(study)
fig2 = optuna.visualization.plot_slice(study)
fig3 = optuna.visualization.plot_param_importances(study)
fig1.show()
fig2.show()
fig3.show()

#for i in range(len(prediction)):
#  print("Optuna Price: {}".format(prediction[i]))

[I 2023-04-16 21:32:24,789] A new study created in memory with name: lightgbm


[LightGBM] [Warning] lambda_l1 is set=30, reg_alpha=0.0 will be ignored. Current value: lambda_l1=30
[LightGBM] [Warning] min_gain_to_split is set=11.360904177316035, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.360904177316035
[LightGBM] [Warning] lambda_l2 is set=55, reg_lambda=0.0 will be ignored. Current value: lambda_l2=55
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 100 rounds


[I 2023-04-16 21:32:25,646] Trial 0 finished with value: 876583143.4029151 and parameters: {'n_estimators': 20000, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.16476035391376567, 'num_leaves': 520, 'max_depth': 19, 'lambda_l1': 30, 'lambda_l2': 55, 'min_gain_to_split': 11.360904177316035, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.5, 'min_data_per_groups': 1}. Best is trial 0 with value: 876583143.4029151.


Early stopping, best iteration is:
[368]	valid_0's rmse: 29607.1
[LightGBM] [Warning] lambda_l1 is set=65, reg_alpha=0.0 will be ignored. Current value: lambda_l1=65
[LightGBM] [Warning] min_gain_to_split is set=8.214765513950292, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=8.214765513950292
[LightGBM] [Warning] lambda_l2 is set=45, reg_lambda=0.0 will be ignored. Current value: lambda_l2=45
[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=0.3 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=0.6 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 100 rounds


[I 2023-04-16 21:32:26,991] Trial 1 finished with value: 805510584.6822413 and parameters: {'n_estimators': 20000, 'colsample_bytree': 0.3, 'subsample': 0.6, 'learning_rate': 0.06934656400876338, 'num_leaves': 1580, 'max_depth': 85, 'lambda_l1': 65, 'lambda_l2': 45, 'min_gain_to_split': 8.214765513950292, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001, 'min_data_per_groups': 52}. Best is trial 1 with value: 805510584.6822413.


...

Early stopping, best iteration is:
[44]	valid_0's rmse: 27562.8
[LightGBM] [Warning] lambda_l1 is set=25, reg_alpha=0.0 will be ignored. Current value: lambda_l1=25
[LightGBM] [Warning] min_gain_to_split is set=0.825359802737851, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.825359802737851
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds


[I 2023-04-16 21:32:39,036] Trial 45 pruned. Trial was pruned at iteration 24.
[I 2023-04-16 21:32:39,117] Trial 46 pruned. Trial was pruned at iteration 0.
[I 2023-04-16 21:32:39,191] Trial 47 pruned. Trial was pruned at iteration 0.


Training until validation scores don't improve for 100 rounds


[I 2023-04-16 21:32:39,684] Trial 48 finished with value: 814150748.4501517 and parameters: {'n_estimators': 20000, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.23892306989780568, 'num_leaves': 980, 'max_depth': 85, 'lambda_l1': 10, 'lambda_l2': 0, 'min_gain_to_split': 3.900707739501007, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.8, 'min_data_per_groups': 21}. Best is trial 27 with value: 732968729.6324134.
[I 2023-04-16 21:32:39,766] Trial 49 pruned. Trial was pruned at iteration 0.


Early stopping, best iteration is:
[51]	valid_0's rmse: 28533.3
[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.0 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] min_gain_to_split is set=2.984740399964145, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.984740399964145
[LightGBM] [Warning] lambda_l2 is set=25, reg_lambda=0.0 will be ignored. Current value: lambda_l2=25
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.7 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
	Best value (rmse): 732968729.63241
	Best params:
		n_estimators: 20000
		colsample_bytree: 0.8
		subsample: 0.7
		learning_rate: 0.230947124846262
		num_leaves: 640
		max_depth: 84
		lambda_l1: 65
		lambda_l2: 0
		min_gain_to_split